In [1]:
from datetime import datetime
from concurrent import futures
import pandas as pd
from pandas import DataFrame
from pathlib import Path
import os
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

In [2]:
""" list of s_anp_p companies """
s_and_p = [
    'MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 
    'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT',
    'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AAPL', 'AMAT', 'APTV', 'ADM',
    'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BBWI', 'BAX', 'BDX', 'BRK.B',
    'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'CHRW',
    'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY',
    'CERN', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX',
    'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CTRA',
    'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DISCA',
    'DISCK', 'DISH', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DUK', 'DRE', 'DD', 'DXC', 'EMN', 'ETN', 'EBAY', 'ECL',
    'EIX', 'EW', 'EA', 'EMR', 'ENPH', 'ETR', 'EOG', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'EVRG', 'ES', 'RE', 'EXC',
    'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FB', 'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FE', 'FRC', 'FISV', 'FLT', 'FMC', 'F',
    'FTNT', 'FTV', 'FBHS', 'FOXA', 'FOX', 'BEN', 'FCX', 'GPS', 'GRMN', 'IT', 'GNRC', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD',
    'GL', 'GPN', 'GS', 'GWW', 'HAL', 'HBI', 'HIG', 'HAS', 'HCA', 'PEAK', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HOLX', 'HD',
    'HON', 'HRL', 'HST', 'HWM', 'HPQ', 'HUM', 'HBAN', 'HII', 'IEX', 'IDXX', 'INFO', 'ITW', 'ILMN', 'INCY', 'IR', 'INTC', 'ICE',
    'IBM', 'IP', 'IPG', 'IFF', 'INTU', 'ISRG', 'IVZ', 'IPGP', 'IQV', 'IRM', 'JKHY', 'J', 'JBHT', 'SJM', 'JNJ', 'JCI', 'JPM',
    'JNPR', 'KSU', 'K', 'KEY', 'KEYS', 'KMB', 'KIM', 'KMI', 'KLAC', 'KHC', 'KR', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LEG','LDOS',
    'LEN', 'LLY', 'LNC', 'LIN', 'LYV', 'LKQ', 'LMT', 'L', 'LOW', 'LUMN', 'LYB', 'MTB', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC',
    'MLM', 'MAS', 'MA', 'MTCH', 'MKC', 'MCD', 'MCK', 'MDT', 'MRK', 'MET', 'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA','MRNA',
    'MHK', 'TAP', 'MDLZ', 'MPWR', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'NDAQ', 'NTAP', 'NFLX', 'NWL', 'NEM', 'NWSA',
    'NWS', 'NEE', 'NLSN', 'NKE', 'NI', 'NSC', 'NTRS', 'NOC', 'NLOK', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'NXPI','ORLY', 'OXY',
    'ODFL', 'OMC', 'OKE', 'ORCL', 'OGN', 'OTIS', 'PCAR', 'PKG', 'PH', 'PAYX', 'PAYC', 'PYPL', 'PENN', 'PNR', 'PBCT', 'PEP',
    'PKI', 'PFE', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'POOL', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PTC', 'PEG', 'PSA',
    'PHM', 'PVH', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTX', 'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RHI', 'ROK', 'ROL',
    'ROP', 'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SEE', 'SRE', 'NOW', 'SHW', 'SPG', 'SWKS', 'SNA', 'SO', 'LUV',
    'SWK', 'SBUX', 'STT', 'STE', 'SYK', 'SIVB', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW', 'TTWO', 'TPR', 'TGT', 'TEL', 'TDY', 'TFX',
    'TER', 'TSLA', 'TXN', 'TXT', 'TMO', 'TJX', 'TSCO', 'TT', 'TDG', 'TRV', 'TRMB', 'TFC', 'TWTR', 'TYL', 'TSN', 'UDR', 'ULTA',
    'USB', 'UAA', 'UA', 'UNP', 'UAL', 'UNH', 'UPS', 'URI', 'UHS', 'VLO', 'VTR', 'VRSN',     'VRSK', 'VZ', 'VRTX', 'VFC', 'VIAC',
    'VTRS', 'V', 'VNO', 'VMC', 'WRB', 'WAB', 'WMT', 'WBA', 'DIS', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WST', 'WDC', 'WU', 'WRK',
    'WY', 'WHR', 'WMB', 'WLTW', 'WYNN', 'XEL', 'XLNX', 'XYL', 'YUM', 'ZBRA', 'ZBH','ZION', 'ZTS'
]

bad_names = []

In [3]:
dirpath = os.path.join(Path().resolve(), 'S&P_500')
Path(dirpath).mkdir(parents=True, exist_ok=True)

In [4]:
now_time = datetime.now()
start_time = datetime(now_time.year - 5, now_time.month , now_time.day)
print(start_time, now_time)

data = pdr.get_data_yahoo('AAPL', start_time, now_time)
data

2018-01-18 00:00:00 2023-01-18 04:53:58.079963
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-18,44.842499,45.025002,44.562500,44.814999,42.614574,124773600
2018-01-19,44.652500,44.895000,44.352501,44.615002,42.424389,129700400
2018-01-22,44.325001,44.445000,44.150002,44.250000,42.077320,108434400
2018-01-23,44.325001,44.860001,44.205002,44.259998,42.086819,130756400
2018-01-24,44.312500,44.325001,43.299999,43.555000,41.416439,204420400
...,...,...,...,...,...,...
2023-01-10,130.259995,131.259995,128.119995,130.729996,130.729996,63896200
2023-01-11,131.250000,133.509995,130.460007,133.490005,133.490005,69458900
2023-01-12,133.880005,134.259995,131.440002,133.410004,133.410004,71379600


In [5]:
def get_stock_data(stock):
    
    """ try to query the iex for a stock, if failed note with print """
    try:
        print(stock)
        stock_df = pdr.get_data_yahoo(stock, start_time, now_time)
        output_name = os.path.join(dirpath, stock) + '.csv'
        stock_df.to_csv(output_name)
    except:
        bad_names.append(stock)
        print('bad: %s' % (stock))
        

In [6]:
# Ребята, не стоит вскрывать эту тему. Вы молодые, шутливые, вам все легко. Это не то.

def get_stock_data_optimal(stock):
    
    """ try to query the iex for a stock, if failed note with print """
    try:
        
        print(stock)
        stock_df = pd.read_csv(os.path.join(Path().resolve(), 'S&P_500', stock) + '.csv', index_col='Date')
        last_date = datetime.strptime(stock_df.tail(1).index.to_list()[0], '%Y-%m-%d')
        now_time = datetime.now()
        start_time = datetime(last_date.year, last_date.month , last_date.day + 1)
        
        if pdr.get_data_yahoo('MMM', start_time, now_time).shape[0] != 0:
            start_time = datetime(last_date.year, last_date.month , last_date.day + 1)
            stock_diff = pdr.get_data_yahoo(stock, start_time, now_time)
            stock_all = pd.concat([stock_df, stock_diff], ignore_index=False, axis=0)            
            cutten_days = abs((now_time - start_time).days) + 1
            stock_all.iloc[cutten_days]
            output_name = os.path.join(dirpath, stock) + '.csv'
            stock_all.to_csv(output_name)
        
    except:
        
        bad_names.append(stock)
        print('bad: %s' % (stock))
        

In [7]:
if __name__ == '__main__':

    """ set the download window """
    now_time = datetime.now()
    start_time = datetime(now_time.year - 1, now_time.month , now_time.day)

    """here we use the concurrent.futures module's ThreadPoolExecutor
        to speed up the downloads buy doing them in parallel 
        as opposed to sequentially """

    #set the maximum thread number
#     max_workers = 50

#     workers = min(max_workers, len(s_and_p)) #in case a smaller number of stocks than threads was passed in
#     with futures.ThreadPoolExecutor(workers) as executor:
#         res = executor.map(get_stock_data, s_and_p)

    for i in s_and_p:
        get_stock_data(i)
    
    """ Save failed queries to a text file to retry """
    if len(bad_names) > 0:
        with open(os.path.join(dirpath, 'failed_queries.txt'),'w') as outfile:
            for name in bad_names:
                outfile.write(name+'\n')

    #timing:
    finish_time = datetime.now()
    duration = finish_time - now_time
    minutes, seconds = divmod(duration.seconds, 60)
    print('bad_names: ', end='')
    print(bad_names)
    print(f'Script took {minutes} minutes and {seconds} seconds to run.')


MMM
[*********************100%***********************]  1 of 1 completed
ABT
[*********************100%***********************]  1 of 1 completed
ABBV
[*********************100%***********************]  1 of 1 completed
ABMD
[*********************100%***********************]  1 of 1 completed
ACN
[*********************100%***********************]  1 of 1 completed
ATVI
[*********************100%***********************]  1 of 1 completed
ADBE
[*********************100%***********************]  1 of 1 completed
AMD
[*********************100%***********************]  1 of 1 completed
AAP
[*********************100%***********************]  1 of 1 completed
AES
[*********************100%***********************]  1 of 1 completed
AFL
[*********************100%***********************]  1 of 1 completed
A
[*********************100%***********************]  1 of 1 completed
APD
[*********************100%***********************]  1 of 1 completed
AKAM
[*********************100%******************

[*********************100%***********************]  1 of 1 completed
CHD
[*********************100%***********************]  1 of 1 completed
CI
[*********************100%***********************]  1 of 1 completed
CINF
[*********************100%***********************]  1 of 1 completed
CTAS
[*********************100%***********************]  1 of 1 completed
CSCO
[*********************100%***********************]  1 of 1 completed
C
[*********************100%***********************]  1 of 1 completed
CFG
[*********************100%***********************]  1 of 1 completed
CTXS
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CTXS: No timezone found, symbol may be delisted
CLX
[*********************100%***********************]  1 of 1 completed
CME
[*********************100%***********************]  1 of 1 completed
CMS
[*********************100%***********************]  1 of 1 completed
KO
[*********************100%***********************]  1 

[*********************100%***********************]  1 of 1 completed
GM
[*********************100%***********************]  1 of 1 completed
GPC
[*********************100%***********************]  1 of 1 completed
GILD
[*********************100%***********************]  1 of 1 completed
GL
[*********************100%***********************]  1 of 1 completed
GPN
[*********************100%***********************]  1 of 1 completed
GS
[*********************100%***********************]  1 of 1 completed
GWW
[*********************100%***********************]  1 of 1 completed
HAL
[*********************100%***********************]  1 of 1 completed
HBI
[*********************100%***********************]  1 of 1 completed
HIG
[*********************100%***********************]  1 of 1 completed
HAS
[*********************100%***********************]  1 of 1 completed
HCA
[*********************100%***********************]  1 of 1 completed
PEAK
[*********************100%***********************]  

[*********************100%***********************]  1 of 1 completed
MNST
[*********************100%***********************]  1 of 1 completed
MCO
[*********************100%***********************]  1 of 1 completed
MS
[*********************100%***********************]  1 of 1 completed
MOS
[*********************100%***********************]  1 of 1 completed
MSI
[*********************100%***********************]  1 of 1 completed
MSCI
[*********************100%***********************]  1 of 1 completed
NDAQ
[*********************100%***********************]  1 of 1 completed
NTAP
[*********************100%***********************]  1 of 1 completed
NFLX
[*********************100%***********************]  1 of 1 completed
NWL
[*********************100%***********************]  1 of 1 completed
NEM
[*********************100%***********************]  1 of 1 completed
NWSA
[*********************100%***********************]  1 of 1 completed
NWS
[*********************100%********************

[*********************100%***********************]  1 of 1 completed
TTWO
[*********************100%***********************]  1 of 1 completed
TPR
[*********************100%***********************]  1 of 1 completed
TGT
[*********************100%***********************]  1 of 1 completed
TEL
[*********************100%***********************]  1 of 1 completed
TDY
[*********************100%***********************]  1 of 1 completed
TFX
[*********************100%***********************]  1 of 1 completed
TER
[*********************100%***********************]  1 of 1 completed
TSLA
[*********************100%***********************]  1 of 1 completed
TXN
[*********************100%***********************]  1 of 1 completed
TXT
[*********************100%***********************]  1 of 1 completed
TMO
[*********************100%***********************]  1 of 1 completed
TJX
[*********************100%***********************]  1 of 1 completed
TSCO
[*********************100%**********************